In [579]:
# clustering with self-organizing-map
class SOM:
    def __init__(self, XX):
        self.epochs = 0
        self.g_epochs = 0
        self.X = XX
        self.samples, features = self.X.shape
        self.w_size = int(np.sqrt(5 * np.sqrt(self.samples)))
        minn = self.X.min()
        maxx = self.X.max()
        self.weights = np.random.uniform(minn, maxx, (self.w_size, self.w_size, features))
        self.eta = 0.5
        self.eta_1 = 0.1
        self.sigma = self.w_size / 2
        self.tau_e = 0.1  # lowest starting learning rate for grand epoch
        self.tau_s = 0.1  # tau for sigma
        self.assignment = {}
        self.clusters = {}
        self.density = None
        
    def train(self, epochs, reduce = True):
        self.g_epochs += 1
        eta = (self.eta - self.eta_1) / self.g_epochs + self.eta_1
        sigma = (self.sigma - 1) / self.g_epochs + 1
        print('eta', eta)
        for epoch in range(epochs):
            sigma = sigma * np.exp(-self.tau_s)
            self.epochs += 1
            #sigma = 1 + (self.sigma - 1) * ((epochs - 1 - epoch) / (epochs - 1))
            eta = eta * self.tau_e**(1 / epochs)
            order = np.random.permutation(self.samples)
            sys.stdout.write('\r[%-50s] %d%%, g_ep %d, ep %d, tot_ep %d' % ('='*int(50*(epoch+1)/epochs), 
                                                100*(epoch+1)/epochs, self.g_epochs, epoch+1, self.epochs))
            sys.stdout.flush()
            for ind in order: 
                dist = self.X[ind] - self.weights
                d_mat = np.einsum('ijk,ijk->ij', dist, dist)
                gh = np.argmin(d_mat, axis=None)
                #ghz = np.argmax(d_mat, axis=None)
                self.assignment[ind] = gh
                #gz, hz = np.unravel_index(ghz, d_mat.shape)
                g, h = np.unravel_index(gh, d_mat.shape)
                
                ii = np.arange(self.w_size) - g
                jj = np.arange(self.w_size) - h
                ii = np.einsum('i,i->i', ii, ii)
                jj = np.einsum('i,i->i', jj, jj)
                ii = eta * ii
                jj = eta * jj
                d_w = (ii * np.ones((self.w_size, self.w_size))).T + jj * np.ones((self.w_size, self.w_size))
                d_w = d_w / (2 * sigma**2)
                d_w = np.exp(-d_w)
                d_w = np.einsum('ijk,ij->ijk', dist, d_w)
                self.weights = self.weights + d_w
        self.get_clusters()
        if reduce:
            self.reduce_neurons()
        sys.stdout.write('\n')
        print('eta', eta)
    
    def if_remotest(g, h, gz, hz):
        if g == gz or h == hz:
            return False
        else:
            if g > gz:
                if gz != 0:
                    return False
            
    def get_clusters(self):
        self.clusters = {}
        self.density = np.zeros((self.w_size, self.w_size))
        for k,v in self.assignment.items():
            self.clusters.setdefault(v, []).append(k)
            i, j = np.unravel_index(v, self.density.shape)
            self.density[i,j] = self.density[i,j] + 1
    
    def reduce_neurons(self):
        for _ in range(2):
            to_remove = self.density.argmin(axis=1)
            new_d = []
            new_w = []
            for i, j in enumerate(to_remove):
                new_d.append(np.delete(self.density[i], j))
                new_w.append(np.delete(self.weights[i], j, axis=0))
            self.density = np.einsum('ij->ji', new_d)
            self.weights = np.einsum('ijk->jik', new_w)
        self.w_size = self.density.shape[0]
    
    def classify(self, x):
        dist = x - self.weights
        d_mat = np.einsum('ijk,ijk->ij', dist, dist)
        return np.argmin(d_mat, axis=None)

# my_som = SOM(XX)
# my_som.train(64)
# while my_som.w_size > 3:
#     my_som.train(32)
# my_som.train(16, False)